In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm
from copy import deepcopy

In [2]:
pd.DataFrame().to_excel('CCR.xlsx')

In [12]:
occ= OccurrenceType.objects.filter(company__name='CCR - Rio Sp',occurrence_kind__in=["42"])
occ.count()

3

In [13]:
# json_data = occ.form_fields

# Function to serialize the specified columns to JSON-formatted strings with double quotation marks
def serialize_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: json.dumps(x))

In [14]:
# fields = json_data['fields']
def get_df(fields,groups,classe):
    columns = ['occ_kind','occ_type','id', 'dataType', 'apiName', 'displayName', 'selectOptions', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
    df = pd.DataFrame(columns=columns)
    groups = groups.get('groups', [])  # Get the groups if available
    
    
    for field in fields:
        field_data = {column: field.get(column, None) for column in columns}

        # Find the group for the field if groups exist
        if groups:
            for group in groups:
                if field_data['id'] in group['members']:
                    field_data['group'] = group['displayName']
                    break
            else:
                field_data['group'] = None
        else:
            field_data['group'] = None

        df=df.append(field_data, ignore_index=True)
    df['occ_kind'] = classe.occurrence_kind
    df['occ_type'] = classe.name
    return df

In [15]:
def options_df_geral(fields,df,classe):
    # Serialize specified columns to JSON-formatted strings with double quotation marks
    serialize_columns(df, ['selectOptions', 'autofill', 'displayIf', 'requiredIf'])

#     output_file = "output.xlsx"  # Specify the output file path and name

#     df.to_excel(output_file, index=False)  # Export DataFrame to Excel
    options_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options']
    options_df = pd.DataFrame(columns=options_columns)
#     teste = deepcopy(options_df)

    for field in fields:
        if field.get('dataType') == 'select' and 'selectOptions' in field:
            display_name = field.get('displayName')
            api_name = field.get('apiName')
            select_options = field.get('selectOptions').get('options', [])

            options = [option.get('name') for option in select_options]

            options_df=options_df.append({'displayName': display_name, 'apiName': api_name, 'options': options}, ignore_index=True)
    options_df['occ_kind'] = classe.occurrence_kind
    options_df['occ_type'] = classe.name
    return options_df
#     options_df.to_excel("options.xlsx", index=False)

In [16]:
def serialized_options_df(options_df,classe):

    # Convert the 'options' column to a list of dictionaries
    options_list = options_df['options'].tolist()

    # Create a new DataFrame with separate columns for each option
    expanded_df = pd.DataFrame(options_list)

    # Concatenate the original columns with the expanded columns
    options_expanded_df = pd.concat([options_df[['occ_kind','occ_type','displayName', 'apiName']], expanded_df], axis=1)
    
    # Adding eighty new columns with names 'option 1' to 'option eighty'
    for i in range(1, 81):
        
        column_name = f'option {i}'
        expanded_df[column_name] = None
    
    # Export the updated options DataFrame to Excel
    # options_expanded_df.to_excel("options_expanded.xlsx", index=False)
    # options_expanded_df['occ_kind'] = classe.occurrence_kind
    # options_expanded_df['occ_type'] = classe.name

    return options_expanded_df


In [17]:
def get_arrayData(fields,classe):
    array_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options', 'order', 'dataType', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
    array_df = pd.DataFrame(columns=array_columns)
    
    for field in fields:

        if field['dataType'] == 'arrayOfObjects':
            inner_fields = field['innerFields']
            for inner_field in inner_fields:
                options = []
                if inner_field['dataType'] == 'select':
                    try:
                        options = [option['name'] for option in inner_field['selectOptions']['options']]
                    except:
                        pass
                inner_field_data = {
                    'displayName': inner_field['displayName'],
                    'apiName': inner_field['apiName'],
                    'options': options,
                    'order': inner_field['order'],
                    'dataType': inner_field['dataType'],
                    'autofill': inner_field.get('autofill', None),
                    'displayIf': inner_field.get('displayIf', None),
                    'requiredIf': inner_field.get('requiredIf', None),
                    'innerfields': inner_field.get('innerfields', None)
                }
                array_df=array_df.append(inner_field_data, ignore_index=True)
            array_df['occ_kind'] = classe.occurrence_kind
            array_df['occ_type'] = classe.name
            return array_df
            

In [18]:
def options_array(fields):  
    # Serialize specified columns to JSON-formatted strings with double quotation marks
    serialize_columns(array_df, ['options', 'autofill', 'displayIf', 'requiredIf'])

    array_df.to_excel("array.xlsx", index=False)

    array_options_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options']
    array_options_expanded = pd.DataFrame(columns=array_options_columns)

    for field in fields:

        if field.get('dataType') == 'arrayOfObjects':
            inner_fields = field.get('innerFields', [])
            for inner_field in inner_fields:
                if inner_field.get('dataType') == 'select':
                    display_name = inner_field.get('displayName')
                    api_name = inner_field.get('apiName')
                    select_options = inner_field.get('selectOptions', {}).get('options', [])

                    options = [option.get('name') for option in select_options]

                    array_options_expanded=array_options_expanded.append({'displayName': display_name, 'apiName': api_name, 'options': options}, ignore_index=True)
    array_options_expanded['occ_kind'] = classe.occurrence_kind
    array_options_expanded['occ_type'] = classe.name
    return array_options_expanded

In [19]:
# def serialized_array(array_options_expanded):    
#     # Serialize specified columns to JSON-formatted strings with double quotation marks
#     serialize_columns(array_options_expanded, ['displayName', 'apiName', 'options'])

#     # Convert the 'options' column to a list of dictionaries
#     options_list = array_options_expanded['options'].tolist()

#     # Get the maximum number of options in any row
#     max_options = max(len(options) for options in options_list)

#     # Create a new DataFrame with separate columns for each option
#     expanded_df = pd.DataFrame(columns=[f"option_{i+1}" for i in range(max_options)])

#     # Fill the expanded DataFrame with the options
#     for options in options_list:
#         row = {}
#         for i, option in enumerate(options):
#             row[f"option_{i+1}"] = option
#         expanded_df = expanded_df.append(row, ignore_index=True)

In [20]:
def generate_final_df(df,odf,array_df,options_expanded_df):
    # Criar DataFrame Final
    
    columns = ['occ_kind','occ_type','id', 'dataType', 'apiName', 'displayName', 'selectOptions', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
    df_final = pd.DataFrame(columns=columns)
    df_final = df_final.append(df)
    
    # Criar DataFrame Final para Options
    options_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options']
    options_df_final = pd.DataFrame(columns=options_columns)
    options_df_final = options_df_final.append(odf)
    
    
    # Criar DataFrame Final para Options Expandida
    options_list = ['occ_kind','occ_type','displayName', 'apiName']
    options_list.append('options_'+i for i in range(80))
    expanded_df = pd.DataFrame(options_list)
    expanded_df = expanded_df.append(options_expanded_df)
    
    
    # Criar DataFrame Final para Array
    array_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options', 'order', 'dataType', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
    array_df_final = pd.DataFrame(columns=array_columns)
    array_df_final = array_df_final.append(array_df)
    return df_final,options_df_final,array_df_final,expanded_df
    

In [21]:
def generate_excel_final(df,odf,array_df,options_expanded_df):    
    # Concatenate the original columns with the expanded columns
#     array_options_expanded = pd.concat([array_options_expanded[['displayName', 'apiName']], expanded_df], axis=1)

    # Export the updated options DataFrame to Excel
#     array_options_expanded.to_excel("array_options_expanded.xlsx", index=False)

    # Save each DataFrame to separate sheets in a single Excel file
    with pd.ExcelWriter('CCR.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name='df', index=False)
        options_expanded_df.to_excel(writer, sheet_name='options_expanded_df', index=False)
        odf.to_excel(writer, sheet_name='options_df', index=False)
        array_df.to_excel(writer, sheet_name='array_df', index=False)
#         array_options_expanded.to_excel(writer, sheet_name='array_options_expanded', index=False)

In [23]:
df_geral = pd.DataFrame()
odf=pd.DataFrame()
options_expanded_df=pd.DataFrame()
df_arrayData=pd.DataFrame()
for a in tqdm(occ):
    try:
        fields = a.form_fields['fields']
        df_geral = pd.concat([df_geral,get_df(fields,a.form_fields,a)],axis=0)
        odf=pd.concat([odf,options_df_geral(fields,df_geral,a)],axis=0)
#         options_expanded_df=pd.concat([options_expanded_df,serialized_options_df(odf,a)],axis=0)
        df_arrayData=pd.concat([df_arrayData,get_arrayData(fields,a)],axis=0)
#         print(df_arrayData)
#         arr=options_array(fields,a)
#         serialized_array(arr)
#         df_final,odf_final,df_arrayData_final,options_expanded_df_final=concat_df(df_geral,odf,df_arrayData,options_expanded_df)
    except Exception as e:
        print(a,e,a.fo)
        pass
generate_excel_final(df_geral,odf,df_arrayData,options_expanded_df)

  0%|          | 0/3 [00:00<?, ?it/s]

In [53]:
bool(options_df.__dict__),options_df

(False, <function __main__.options_df(fields, df)>)

In [35]:
for field in fields:

        if field['dataType'] == 'arrayOfObjects':
            inner_fields = field['innerFields']
            for inner_field in inner_fields:
                options = []
                if inner_field['dataType'] == 'select':
                    try:
                        options = [option['name'] for option in inner_field['selectOptions']['options']]
                    except:
                        pass

In [40]:
for a in tqdm(occ):
    print(a.occurrence_kind,a.name)

  0%|          | 0/238 [00:00<?, ?it/s]

44 Substituição de elementos na faixa de domínio
48 Reparo na base/sapata/bloco
56 Despraguejamento
49 Remoção de Pintura por abrasão
44 Cata Papel
2 Drenagem Superficial
41 Retrorrefletância de Dispositivos
56 Conformação / Regularização de terreno
41 Retrorrefletância Horizontal Longitudinal
41 Retrorrefletância Horizontal de Zebrado
47 Remoção  EPS
36 Monitoração de Barreira New Jersey
33 Monitoração de Drenagem Profunda
47 Remoção e reimplantação do tramo EPS
49 Instalação de tachas e tachões
55 Reparo Placa de Concreto
35 Monitoração de Edificações SGE
51 Implantação de elemento de OAE
38 Monitoração Degrau
56 Plantio de Vetiver
41 Monitoração de Sinalização Vertical
44 Reforço estrutural de elementos na faixa de domínio
2 Sinalização Horizontal (Zebrado)
46 Recuperação Terrapleno
38 Monitoração Pêndulo Britânico
44 Controle fitosanitário - erradicação
56 Enleivamento
38 Monitoração IRI Total
44 Desocupação
51 Fixação do elemento de OAE
44 Implantação de elementos na faixa de domí

In [45]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

In [115]:
df.columns

Index(['A', 'B', 'C'], dtype='object')